In [1]:
# Import Library
import os,csv
import pandas as pd
from tqdm import trange
import copy
import numpy as np

### (Train Set) Load "challenge-2018-train-annotations-bbox.csv" 

In [2]:
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
data = pd.read_csv('Excel Files/challenge-2018-train-annotations-bbox.csv',low_memory=False)
# Obtain necessary columns and generate width&height using min&max
ImageID_train = copy.copy(data['ImageID'])
for  i in trange(len(ImageID_train)):
    ImageID_train[i] = ImageID_train[i]+".txt"
# Load label data and map label names to numeric labels to be fed to darknet YOLO v3 training
# Load data to carry out label mapping & convert it to dictionary
LabelName_train = copy.copy(data['LabelName'])
LabelName_train_kaggle = copy.copy(data['LabelName'])
LabelName_train_mapping = pd.read_csv('challenge-2018-class-descriptions-500.csv')
LabelName_train_mapping_dict = dict()
for i in range(len(LabelName_train_mapping)):
      LabelName_train_mapping_dict[LabelName_train_mapping['kaggle_label'][i]] = LabelName_train_mapping['darknet_label'][i]
# Use dictionary to convert labels in "LabelName_train" to its numeric equivalent labels for darknet training
for i in trange(len(LabelName_train)):
    LabelName_train[i] = LabelName_train_mapping_dict[LabelName_train[i]]
XMin_train = copy.copy(data['XMin'])
YMin_train = copy.copy(data['YMin'])
Width_train = copy.copy(abs(data['XMax']-data['XMin']))
Height_train = copy.copy(abs(data['YMax']-data['YMin']))
XCenter_train = XMin_train + Width_train/2
# Darknet uses left-top as zero coordinate. Hence, 1-YCenter_train
YCenter_train = 1-(YMin_train + Height_train/2)

100%|███████████████████████████████████████████████████████████████████| 12195144/12195144 [06:14<00:00, 32589.19it/s]


### (Train Set) Create log text file so you don't have to run "In [2]:" everytime

In [26]:
# # Combine texts to output in "<object-class> <x_center> <y_center> <width> <height>"
darknet_input = []
for i in trange(len(LabelName_train)):
    darknet_input.append( (str(LabelName_train[i])+" "+str(XCenter_train[i])+
                        " "+str(YCenter_train[i])+" "+str(Width_train[i])+
                        " "+str(Height_train[i]) ))
# Save final input to a txt file
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
with open('darknet_input.txt','w') as f:
    for lines in darknet_input:
        f.write(lines)
        f.write('\n')

### (Train Set) Delete contents in text files in main "train" folder

In [4]:
ImageID_train_set = copy.copy((ImageID_train))
ImageID_train_set = set(ImageID_train_set)                         
ImageID_train_set = list(ImageID_train_set)
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/train")
for i in trange(len(ImageID_train_set)):
    open(ImageID_train_set[i],'w').close()

100%|██████████████████████████████████████████████████████████████████████| 1674979/1674979 [09:31<00:00, 2932.53it/s]


### (Train Set) Create individual text files with "(object-class) (x_center) (y_center) (width) (height)" per darknet specification in destination folder

In [5]:
# import "darknet_input.txt" if "In [2]:" was not ran
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
darknet_input = []
with  open("darknet_input.txt") as f:
    darknet_input = f.readlines()
    
for i in trange(len(darknet_input)):
    darknet_input[i] = darknet_input[i][:-1]

data = pd.read_csv('Excel Files/challenge-2018-train-annotations-bbox.csv',low_memory=False)
# Obtain necessary columns and generate width&height using min&max
ImageID_train = copy.copy(data['ImageID'])
for  i in trange(len(ImageID_train)):
    ImageID_train[i] = ImageID_train[i]+".txt"
print(len(ImageID_train))
print(len(darknet_input))
# Specify folder to save text files to
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/train")
for i in trange(len(darknet_input)):
    with open(ImageID_train[i],'a') as f:
        f.write(darknet_input[i])



12195144
12195144


100%|███████████████████████████████████████████████████████████████████| 12195144/12195144 [9:54:28<00:00, 341.90it/s]


### (Validation Set) Create "real_validation-annotations-bbox.csv" from original annotation file by taking out classes not included in the competition.
(Validation annotation contained classes outside 500 classes from competition and didn't allow me to easily map kaggle's labels to darknet's labels. ex) "/m/061hd_"(Kaggle) -> "0" (Darknet))

In [ ]:
# Import original annotation data provided by Kaggle competition
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
data_val = pd.read_csv('Excel Files/validation-annotations-bbox.csv',low_memory=False)
#Set up mapping
LabelName_val_mapping = pd.read_csv('challenge-2018-class-descriptions-500.csv')
LabelName_val_mapping_dict = dict()
for i in range(len(LabelName_val_mapping)):
      LabelName_val_mapping_dict[LabelName_val_mapping['kaggle_label'][i]] = LabelName_val_mapping['darknet_label'][i]
# Pop the row that is not within 500 categories from the competition
for i in trange(len(data_val['LabelName'])):
    if data_val['LabelName'][i] in LabelName_val_mapping_dict:
        pass
    else  :
        data_val = data_val.drop(i)

### (Validation Set) Delete contents in text files in main "val" folder

In [6]:
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data")
data_val = pd.read_csv('real_validation-annotations-bbox.csv',low_memory=False)
# Obtain necessary columns and generate width&height using min&max
ImageID_val = copy.copy(data_val['ImageID'])
for  i in range(len(ImageID_val)):
    ImageID_val[i] = ImageID_val[i]+".txt"

ImageID_val_set = copy.copy((ImageID_val))
ImageID_val_set = set(ImageID_val_set)                         
ImageID_val_set = list(ImageID_val_set)
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/val")
for i in trange(len(ImageID_val_set)):
    open(ImageID_val_set[i],'w').close()

100%|██████████████████████████████████████████████████████████████████████████| 33073/33073 [00:06<00:00, 5317.74it/s]


### (Validation Set) Perform all of the above for validation set

In [7]:
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data")
data_val = pd.read_csv('real_validation-annotations-bbox.csv',low_memory=False)
# Obtain necessary columns and generate width&height using min&max
ImageID_val = copy.copy(data_val['ImageID'])
for  i in range(len(ImageID_val)):
    ImageID_val[i] = ImageID_val[i]+".txt"
# Load label data and map label names to numeric labels to be fed to darknet YOLO v3 training
# Load data to carry out label mapping & convert it to dictionary
LabelName_val = copy.copy(data_val['LabelName'])
LabelName_val_mapping = pd.read_csv('challenge-2018-class-descriptions-500.csv')
LabelName_val_mapping_dict = dict()
for i in range(len(LabelName_val_mapping)):
      LabelName_val_mapping_dict[LabelName_val_mapping['kaggle_label'][i]] = LabelName_val_mapping['darknet_label'][i]
# Use dictionary to convert labels in "LabelName_train" to its numeric equivalent labels for darknet training
for i in trange(len(LabelName_val)):
    LabelName_val[i] = LabelName_val_mapping_dict[LabelName_val[i]]
XMin_val = copy.copy(data_val['XMin'])
YMin_val = copy.copy(data_val['YMin'])
Width_val = copy.copy(abs(data_val['XMax']-data_val['XMin']))
Height_val = copy.copy(abs(data_val['YMax']-data_val['YMin']))
XCenter_val = XMin_val + Width_val/2
YCenter_val = 1-(YMin_val + Height_val/2)

# Combine texts to output in "<object-class> <x_center> <y_center> <width> <height>"
darknet_input_val = []
for i in trange(len(LabelName_val)):
    darknet_input_val.append( (str(LabelName_val[i])+" "+str(XCenter_val[i])+
                        " "+str(YCenter_val[i])+" "+str(Width_val[i])+
                        " "+str(Height_val[i]) ))
# Save final input to a txt file
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
with open('darknet_input_val.txt','w') as f:
    for lines in darknet_input_val:
        f.write(lines)
        f.write('\n')


# # import "darknet_input_val.txt" if "In [2]:" was not ran
# darknet_input_val = []
# with  open("darknet_input_val.txt") as f:
#     darknet_input_val = f.readlines()
# Specify folder to save text files to.
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/val")
for i in trange(len(ImageID_val)):
    with open(ImageID_val[i],'a') as f:
        f.write(darknet_input_val[i])
        # Don't include below line if reading info from 
        f.write('\n')
        


100%|█████████████████████████████████████████████████████████████████████████| 166905/166905 [07:51<00:00, 353.63it/s]


### (Train Set) Create "train.txt" file to be put into train folder for darknet

In [23]:
# Import train data
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
data_train = pd.read_csv('Excel Files/challenge-2018-train-annotations-bbox.csv',low_memory=False)
# Obtain just Image names
ImageID_train = copy.copy(data_train['ImageID'])
# Remove duplicate
ImageID_train = set(ImageID_train)
# Turn data into list again
ImageID_train = list(ImageID_train)
# train.txt file to be put into train folder for darknet training
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/train")
with open("train.txt",'w') as f:
    for lines in ImageID_train:
        f.write("data/train/")
        f.write(lines)
        f.write(".jpg")
        f.write('\n')

### (Validation Set) Create "val.txt" file to be put into train folder for darknet

In [24]:
#Import train data
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
data_val = pd.read_csv('Excel Files/real_validation-annotations-bbox.csv',low_memory=False)
# Obtain just Image names
ImageID_val = copy.copy(data_val['ImageID'])
# Remove duplicate
ImageID_val = set(ImageID_val)
# Turn data into list again
ImageID_val = list(ImageID_val)
# val.txt file to be put into train folder for darknet training
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/val")
with open("val.txt",'w') as f:
    for lines in ImageID_val:
        f.write("data/val/")
        f.write(lines)
        f.write(".jpg")
        f.write('\n')

### (Test) Add directory info to "test.txt"

In [20]:
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/test/challenge2018_test")
test_text = os.listdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/test/challenge2018_test")


In [21]:
for i in range(len(test_text)):
    test_text[i] = "data/test/challenge2018_test/"+test_text[i]

In [22]:
with open("test.txt",'w') as f:
    for lines in test_text:
        f.write(lines)
        f.write('\n')

### (Test) Submission File - Process Darknet's output in a .txt to .csv

In [3]:
# Read results from darknet prediction output
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64")
submission_text = []
with open("result_coco.txt") as f:
    submission_text = f.readlines()
# Trim "\n" from end of each line from text file
for i in trange(len(submission_text)):
    submission_text[i] = submission_text[i][:-2]
# Save image name and prediction string separately  
submission_ImageId = []
submission_pred = []
for i in trange(len(submission_text)):
    submission_ImageId.append(submission_text[i][10:26])
    submission_pred.append(submission_text[i][31:])
# # Replaces wrongly labeled label
# for i in trange(len(submission_pred)):
#     if "traffic light" in submission_pred[i]:
#         submission_pred[i]=submission_pred[i].replace('traffic light','/m/015qff')
# Save "submission_ImageId" and "submission_pred" under correct headers for final submission
raw_df = pd.DataFrame({"ImageId":submission_ImageId,"PredictionString":submission_pred})
raw_df.to_csv("submission_coco.csv",index=False)

100%|███████████████████████████████████████████████████████████████████████| 99999/99999 [00:00<00:00, 1542575.23it/s]


###  (Test) Create "test.txt" file and add appropriate directory path

In [2]:
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/test")
filenames = os.listdir()
print(len(filenames))
for i in trange(len(filenames)):
    filenames[i]= "data/test/"+filenames[i]
with open("test.txt",'w') as f:
    for lines in filenames:
        f.write(lines)
        f.write('\n')
    

99999

### (Train) Create testing environment for darknet without labels covered by COCO

In [2]:
#Read training annotation files
train_file_entry = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/Excel Files/challenge-2018-train-annotations-bbox.csv")
# Read COCO mapping
coco_mapping = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/Excel Files/coco_mapping.csv")
coco_mapping_dict = dict()
for i in range(len(coco_mapping)):
    coco_mapping_dict[coco_mapping['label'][i]] = coco_mapping['name'][i]

In [2]:
# Instead of popping...append!
train_ImageID = []
train_LabelName = []
train_XMin = []
train_XMax = []
train_YMin = []
train_YMax = []
print("Original entry length : ",len(train_file_entry))
for i in trange(len(train_file_entry)):
    if train_file_entry['LabelName'][i] not in coco_mapping_dict:
        train_ImageID.append(train_file_entry['ImageID'][i])
        train_LabelName.append(train_file_entry['LabelName'][i])
        train_XMin.append(train_file_entry['XMin'][i])
        train_XMax.append(train_file_entry['XMax'][i])
        train_YMin.append(train_file_entry['YMin'][i])
        train_YMax.append(train_file_entry['YMax'][i])


In [5]:
# Save new data to a csv file
print("New entry length : ",len(train_XMin))
# Write new csv
train_df = pd.DataFrame({"ImageID":train_ImageID,"LabelName":train_LabelName,"XMin":train_XMin,"XMax":train_XMax,"YMin":train_YMin,"YMax":train_YMax})
train_df.to_csv("train_annotation_without_coco.csv",index=False)
print("It is saved in :",os.getcwd())

New entry length :  10114966
It is saved in : E:\Machine Learning\Kaggle - Open Images Challenge


In [6]:
# Load new data to a csv without data covered by COCO
train_df = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/train_annotation_without_coco.csv")

In [6]:
# Delete all .txt files
directory_path = "E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/train"
os.chdir(directory_path)
files_to_delete=os.listdir(directory_path)

for i in trange(len(files_to_delete)):
    if files_to_delete[i].endswith(".txt"):
        os.remove(files_to_delete[i])

In [7]:
ImageID_train = copy.copy(np.asarray(train_df['ImageID']))
for  i in range(len(ImageID_train)):
    ImageID_train[i] = ImageID_train[i]+".txt"
# Load data to carry out label mapping & convert it to dictionary
LabelName_train = copy.copy(train_df['LabelName'])

LabelName_train_mapping = list(set(LabelName_train))
LabelName_train_mapping_dict = dict()
for i in range(len(LabelName_train_mapping)):
      LabelName_train_mapping_dict[LabelName_train_mapping[i]] = i
# Use dictionary to convert labels in "LabelName_train" to its numeric equivalent labels for darknet training
for i in trange(len(LabelName_train)):
    LabelName_train[i] = LabelName_train_mapping_dict[LabelName_train[i]]
XMin_train = copy.copy(train_df['XMin'])
YMin_train = copy.copy(train_df['YMin'])
Width_train = copy.copy(abs(train_df['XMax']-train_df['XMin']))
Height_train = copy.copy(abs(train_df['YMax']-train_df['YMin']))
XCenter_train = XMin_train + Width_train/2
YCenter_train = YMin_train + Height_train/2

# Combine texts to output in "<object-class> <x_center> <y_center> <width> <height>"
darknet_input_train = []
for i in trange(len(LabelName_train)):
    darknet_input_train.append( (str(LabelName_train[i])+" "+str(XCenter_train[i])+
                        " "+str(YCenter_train[i])+" "+str(Width_train[i])+
                        " "+str(Height_train[i]) ))
# Save final input to a txt file
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
with open('darknet_input_val_aug_23.txt','w') as f:
    for lines in darknet_input_train:
        f.write(lines)
        f.write('\n')


# import "darknet_input_val_aug_23.txt" if "In [2]:" was not ran
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
darknet_input_train = []
with  open("darknet_input_train_aug_23.txt") as f:
    darknet_input_train = f.readlines()

# Specify folder to save text files to.
train_df = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/train_annotation_without_coco.csv")
ImageID_train = copy.copy(np.asarray(train_df['ImageID']))
for  i in range(len(ImageID_train)):
    ImageID_train[i] = ImageID_train[i]+".txt"
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/train")
for i in trange(len(ImageID_train)):
    with open(ImageID_train[i],'a') as f:
        f.write(darknet_input_train[i])
        # Don't include below line if reading info from text file
        #f.write('\n')
        


100%|███████████████████████████████████████████████████████████████████| 10114966/10114966 [8:17:43<00:00, 338.71it/s]


### (Val) Create testing environment for darknet without labels covered by COCO

In [2]:
#Read training annotation files
val_file_entry = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/Excel Files/real_validation-annotations-bbox.csv")
# Read COCO mapping
coco_mapping = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/Excel Files/coco_mapping.csv")
coco_mapping_dict = dict()
for i in range(len(coco_mapping)):
    coco_mapping_dict[coco_mapping['label'][i]] = coco_mapping['name'][i]

In [7]:
# # Instead of popping...append!
# val_ImageID = []
# val_LabelName = []
# val_XMin = []
# val_XMax = []
# val_YMin = []
# val_YMax = []
# print("Original entry length : ",len(val_file_entry))
# for i in trange(len(val_file_entry)):
#     if val_file_entry['LabelName'][i] not in coco_mapping_dict:
#         val_ImageID.append(val_file_entry['ImageID'][i])
#         val_LabelName.append(val_file_entry['LabelName'][i])
#         val_XMin.append(val_file_entry['XMin'][i])
#         val_XMax.append(val_file_entry['XMax'][i])
#         val_YMin.append(val_file_entry['YMin'][i])
#         val_YMax.append(val_file_entry['YMax'][i])

Original entry length :  166905


100%|███████████████████████████████████████████████████████████████████████| 166905/166905 [00:08<00:00, 20048.04it/s]


In [8]:
# # Save new data to a csv file
# print("New entry length : ",len(val_XMin))
# # Write new csv
# val_df = pd.DataFrame({"ImageID":val_ImageID,"LabelName":val_LabelName,"XMin":val_XMin,"XMax":val_XMax,"YMin":val_YMin,"YMax":val_YMax})
# val_df.to_csv("val_annotation_without_coco.csv",index=False)
# print("It is saved in :",os.getcwd())

New entry length :  129722
It is saved in : E:\Machine Learning\Kaggle - Open Images Challenge


In [3]:
# Load new data to a csv without data covered by COCO
val_df = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/val_annotation_without_coco.csv")

In [3]:
# # Delete .txt files
# directory_path = "E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/val"
# os.chdir(directory_path)
# files_to_delete=os.listdir(directory_path)

# for i in trange(len(files_to_delete)):
#     if files_to_delete[i].endswith(".txt"):
#         os.remove(files_to_delete[i])

100%|█████████████████████████████████████████████████████████████████████████| 68800/68800 [00:03<00:00, 18113.44it/s]


In [4]:
# Load data to carry out label mapping & convert it to dictionary
LabelName_val = copy.copy(val_df['LabelName'])

LabelName_val_mapping = list(set(LabelName_val))
LabelName_val_mapping_dict = dict()
for i in range(len(LabelName_val_mapping)):
      LabelName_val_mapping_dict[LabelName_val_mapping[i]] = i

In [4]:
ImageID_val = copy.copy(val_df['ImageID'])
for  i in range(len(ImageID_val)):
    ImageID_val[i] = ImageID_val[i]+".txt"
# Load data to carry out label mapping & convert it to dictionary
LabelName_val = copy.copy(val_df['LabelName'])

LabelName_val_mapping = list(set(LabelName_val))
LabelName_val_mapping_dict = dict()
for i in range(len(LabelName_val_mapping)):
      LabelName_val_mapping_dict[LabelName_val_mapping[i]] = i
# Use dictionary to convert labels in "LabelName_train" to its numeric equivalent labels for darknet training
for i in trange(len(LabelName_val)):
    LabelName_val[i] = LabelName_val_mapping_dict[LabelName_val[i]]
XMin_val = copy.copy(val_df['XMin'])
YMin_val = copy.copy(val_df['YMin'])
Width_val = copy.copy(abs(val_df['XMax']-val_df['XMin']))
Height_val = copy.copy(abs(val_df['YMax']-val_df['YMin']))
XCenter_val = XMin_val + Width_val/2
YCenter_val = YMin_val + Height_val/2

# Combine texts to output in "<object-class> <x_center> <y_center> <width> <height>"
darknet_input_val = []
for i in trange(len(LabelName_val)):
    darknet_input_val.append( (str(LabelName_val[i])+" "+str(XCenter_val[i])+
                        " "+str(YCenter_val[i])+" "+str(Width_val[i])+
                        " "+str(Height_val[i]) ))
# Save final input to a txt file
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
with open('darknet_input_val_aug_23.txt','w') as f:
    for lines in darknet_input_val:
        f.write(lines)
        f.write('\n')


# import "darknet_input_val.txt" if "In [2]:" was not ran
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge")
darknet_input_val = []
with  open("darknet_input_val.txt") as f:
    darknet_input_val = f.readlines()
val_df = pd.read_csv("E:/Machine Learning/Kaggle - Open Images Challenge/val_annotation_without_coco.csv")
ImageID_val = copy.copy(np.asarray(val_df['ImageID']))
for  i in range(len(ImageID_val)):
    ImageID_val[i] = ImageID_val[i]+".txt"
# Create individual txt files for each image
os.chdir("E:/Machine Learning/Kaggle - Open Images Challenge/darknet/build/darknet/x64/data/val")
for i in trange(len(ImageID_val)):
    with open(ImageID_val[i],'a') as f:
        f.write(darknet_input_val[i])
        # Don't include below line if reading info from text file
        #f.write('\n')
        


100%|███████████████████████████████████████████████████████████████████████| 129722/129722 [00:05<00:00, 23116.74it/s]
